In [1]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time
import torch

import pymorphy2
import re
import spacy

from string import punctuation

data_path = "data/"

import os
print(os.cpu_count())

64


In [2]:
df = pd.read_parquet(data_path+"keywords/df_keywords.parquet.gzip")
print(df.shape)

(1811222, 13)


In [3]:
with open(data_path+"embeddings_dict.pkl", "rb") as pickle_file:
    embeddings_dict =  pickle.load(pickle_file)
print(len(embeddings_dict))

df = df[~df.keywords.isin(embeddings_dict)].reset_index(drop=True)
print(df.shape)

60099
(1472888, 13)


In [ ]:
model =  SentenceTransformer('intfloat/multilingual-e5-large')

batch_size = 50
total_batches = (len(df) + batch_size - 1) // batch_size

for i in tqdm(range(total_batches), desc="Processing Batches"):
    dfc = df[i*batch_size:(i+1)*batch_size].reset_index(drop=True)
    toEmbed = ["query: " + sent for sent in dfc.keywords.to_list()]
    embeddings_dict.update((dict(zip(dfc.keywords, model.encode(toEmbed)))))
    if i % 10 == 0:
        with open(data_path+"embeddings_dict.pkl", "wb") as pickle_file:
            pickle.dump(embeddings_dict, pickle_file)

Processing Batches:   2%|▏         | 562/29458 [5:37:55<291:51:19, 36.36s/it]

In [ ]:
print(1)